In [1]:
import simpy
import random
from random import seed

In [2]:
def customer(env, name, restaurant, arrival_rate, **duration):
    while True:

        # change 1: customer arrival follows poisson distribution
#        yield env.timeout(random()*10)
        interarrival_time = random.expovariate(arrival_rate)
        yield env.timeout(interarrival_time)

        print(f'{name} enters the restaurant and for the waiter to come at {round(env.now, 2)}')
        with restaurant.request() as req:
            yield req

            print(f"Sits are available. {name} get sitted at {round(env.now, 2)}")
            yield env.timeout(duration['get_sitted'])

            print(f"{name} starts looking at the menu at {round(env.now, 2)}")
            yield env.timeout(duration['choose_food'])

            print(f'Waiters start getting the order from {name} at {round(env.now, 2)}')
            yield env.timeout(duration['give_order'])

            # change 2: food department from kitchen follows poisson distribution
            print(f'{name} starts waiting for food at {round(env.now, 2)}')
#            yield env.timeout(duration['wait_for_food'])
            serving_departure_rate = random.expovariate(1/duration['wait_for_food'])
            yield env.timeout(serving_departure_rate)

            print(f'{name} starts eating at {round(env.now, 2)}')
            yield env.timeout(duration['eat'])

            print(f'{name} starts paying at {round(env.now, 2)}')
            yield env.timeout(duration['pay'])

            print(f'{name} leaves at {round(env.now, 2)}')

In [3]:
seed(1)
env = simpy.Environment()

# Model restaurant that can only allow 2 customers at once
restaurant = simpy.Resource(env, capacity=4)
durations = {'get_sitted': 1, 'choose_food': 10, 'give_order': 5, 'wait_for_food': 20, 'eat': 45, 'pay': 10}


customer_arrival_rate = 10/60 # num customers per hour

for i in range(5):
    env.process(customer(env, f'Customer {i}', restaurant, customer_arrival_rate, **durations))

env.run(until=95)


Customer 0 enters the restaurant and for the waiter to come at 0.87
Sits are available. Customer 0 get sitted at 0.87
Customer 3 enters the restaurant and for the waiter to come at 1.77
Sits are available. Customer 3 get sitted at 1.77
Customer 0 starts looking at the menu at 1.87
Customer 3 starts looking at the menu at 2.77
Customer 4 enters the restaurant and for the waiter to come at 4.1
Sits are available. Customer 4 get sitted at 4.1
Customer 4 starts looking at the menu at 5.1
Customer 2 enters the restaurant and for the waiter to come at 8.66
Sits are available. Customer 2 get sitted at 8.66
Customer 2 starts looking at the menu at 9.66
Customer 1 enters the restaurant and for the waiter to come at 11.28
Waiters start getting the order from Customer 0 at 11.87
Waiters start getting the order from Customer 3 at 12.77
Waiters start getting the order from Customer 4 at 15.1
Customer 0 starts waiting for food at 16.87
Customer 3 starts waiting for food at 17.77
Waiters start gettin